"""
The MIT License (MIT)

Copyright (c) 2017 Chris Scott

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF
MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT.
IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM,
DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE
OR OTHER DEALINGS IN THE SOFTWARE.
"""

<a href="https://colab.research.google.com/github/ABMI-Kevin/Recognizers_OVEN-test/blob/main/OVEN_22022017_upgrade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Packages

Import packages to run the model.  Updates to packages since the model was originally created may result in warnings when running the model.  As of March 2024 the most up to date versions of the packages below allowed the model to be run successfully on audio data despite any warnings that python may generate in the end.

In [19]:
from datetime import timedelta
import librosa
import numpy as np
import os
import pandas as pd
from scipy.signal import butter, filtfilt
from scipy.special import expit
import tensorflow as tf
import time
from tqdm import tqdm_notebook


## Designate Recording Directory

Enter the full file path of the folder where the recording files to be analyzed are stored.

Result files will be populated in a subfolder named "results" in this same directory.

In [2]:
# Path to folder of recordings for analysis
DIRECTORY = 'C:/Users/Kevin Kelly/Documents/Repos/Recognizers_OVEN-test/Recordings_OVEN' # change me!

## Run the Model

The following script will pull the Ovenbird detection model into python and run the audio files through it.  

In [20]:
def highpass_filter(y, fc, sr):
    nyq = 0.5 * sr
    b, a = butter(4, fc/nyq, 'high')
    return filtfilt(b, a, y)

def normalise_audio(y, sr=16000):
    y = highpass_filter(y=y, fc=500.0, sr=sr)
    y -= np.mean(y)
    return y

def spectrogram_from_audio(y, sr):
    sr = 16000
    n_fft = 128
    hop_length = 64
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length, window=np.blackman)[10:,:] # remove DC
    D = librosa.amplitude_to_db(np.abs(D)**2, ref=1.0)
    D = np.swapaxes(D,0,1)[np.newaxis,np.newaxis,:,:] # [batch,height,width,channels]
    return D

def process_directory(directory):
    # make sure there is somewhere to save results
    results_directory = os.path.join(directory, 'results')
    if not os.path.exists(results_directory):
        os.makedirs(results_directory)

    # find tensorflow model files
    meta_graph_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'oven-model-22022017.meta')
    checkpoint_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), 'oven-model-22022017')

    # list all audio files in directory
    audio_files = librosa.util.find_files(directory, ext=['flac', 'wav'], recurse=True, case_sensitive=False, limit=None, offset=0)
    audio_files = list(set(audio_files)) # remove duplicates - not sure why find_files produces them!

    tf.compat.v1.reset_default_graph()
    with tf.compat.v1.Session() as sess:
        new_saver = tf.compat.v1.train.import_meta_graph(meta_graph_path)
        new_saver.restore(sess, checkpoint_path)
        features = tf.compat.v1.get_collection('input')[0]
        is_training = tf.compat.v1.get_collection('is_training')[0]
        level = tf.compat.v1.get_collection('level')[0]
        detection_fn = tf.compat.v1.get_collection('detection_fn')[0]

        for filename in tqdm_notebook(audio_files):
            y, sr = librosa.load(filename, sr=16000, mono=True)
            y = normalise_audio(y)
            S = spectrogram_from_audio(y, sr)
            level_dB, detection_function = sess.run([level, detection_fn], feed_dict={features: S, is_training: False})
            level_dB = level_dB[0,0,:,0]
            detection_function = detection_function[0,0,:,0]
            detection_function = expit(detection_function)
            duration_secs = y.size / float(sr)
            times = np.interp(np.arange(detection_function.size), [0,detection_function.size-1], [0,duration_secs])
            df = pd.DataFrame({'0': times, '1': detection_function, '2': level_dB})

            base = os.path.basename(filename)
            filename = os.path.splitext(base)[0]
            savename = os.path.join(results_directory, filename + '_detection_function.csv')
            df.to_csv(savename, header=False, index=False)

process_directory(DIRECTORY)

'local_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
'model_variables' collection should be of type 'byte_list', but instead is of type 'node_list'.
INFO:tensorflow:Restoring parameters from c:\Users\Kevin Kelly\Documents\Repos\Recognizers_OVEN-test\oven-model-22022017


C:\Users\Kevin Kelly\AppData\Local\Temp\ipykernel_22472\1789341417.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for filename in tqdm_notebook(audio_files):


  0%|          | 0/784 [00:00<?, ?it/s]